# College Football - Pull Data

The goal of this notebook is to pull in all the data from CollegeFootballData.com and export it to the data folder. 

#### Before running this notebook 

* Get custom API key emailed here: https://collegefootballdata.com/key
* Add it to a config/api_key.json file under the key api_key

##### Datasets extracted

1. **Recruiting information:** This data contians infromatioon about the high schooleers that were rectuited into college football between a certain timeframe. 

2. **Team:** This data contains all the college teams in the dataset. 

3. **Game:** This data contains all the the games that have been playbed between a certain timeframe. Each row is oen game

5. **Roster:** For every single team from 2013 - 2023, get all their roster data from the API.

6. **Team Composite Ratings:** For every single team in every year, pull in their team composite rating from the same API. This metric looks at all the players on the roster, and aggregates their recruting profile.

7. **Game Manipulated:** This data takes in the game dataset pulled above and manipulates it from one row is one game, and instead create two records from one game: one from the perspective of the home team and one for the away team. This makes it easier for analysis and offers easy analysis on more familiar metrics like points for, etc.

#### ----------------------------------

###### Helpful Tutorial
https://blog.collegefootballdata.com/introduction-to-cfb-analytics/

###### Actual Documentation
https://api.collegefootballdata.com/api/docs/?url=/api-docs.json


In [2]:
# Uncomment and run line below if cfbd library isn't already installed
#! pip install cfbd
from IPython.display import clear_output

import cfbd
import numpy as np
import pandas as pd
import json
import cfbd

pd.set_option('display.max_columns', None)
clear_output()


## Set up api connection

In [3]:
# Running this code by itself won't work. You'll need your own API Key.
# See link above to have custom API link emailed and save that key as variable api_key.

# Load JSON data from file
with open('../config/api_key.json', 'r') as file:
    data = json.load(file)

# Get the value of 'api_key'
api_key = data.get('api_key')

In [4]:
def api_setup(api_key):

    """
    Configure the api. 
    Only input is the apikey which can be created from the link above.
    """
    import cfbd
    
    configuration = cfbd.Configuration()
    configuration.api_key['Authorization'] = api_key
    configuration.api_key_prefix['Authorization'] = 'Bearer'

    return cfbd.ApiClient(configuration)
    
api_config = api_setup(api_key)

## Define timeframe 

In [5]:
start_year_timeframe = 2015
end_year_timeframe = 2023

## Get Datasource 1 - Player Recruiting Rankings

Get each football players ranking and origin information as they were recruited into college each year. 

In [6]:
def hs_recruits(start_year, end_year):
    
    """
    Two inputs: start_year and end_year (the ranges of years we want the recruiting data for - inclusive)
    
    1) Get each year as a json
    2) Convert to df
    3) Union each year's df together.
    """

    recruits_df_list = []

    for i in range(start_year, end_year + 1):

        # Connect to api for given year
        recr_api = cfbd.RecruitingApi(api_config)
        recruits = recr_api.get_recruiting_players(year = i)

        # Convert json to df
        df_recruits = pd.DataFrame.from_records([r.to_dict() for r in recruits])

        # Append dfs together to create list of dfs
        recruits_df_list.append(df_recruits)

    # Concatenate / union each year's df together
    df_recruits_final = pd.concat(recruits_df_list).reset_index()
    
    df_recruits_final['latitude'] = df_recruits_final.hometown_info.str['latitude']
    df_recruits_final['longitude'] = df_recruits_final.hometown_info.str['longitude']
    
    df_recruits_final.drop(columns = 'hometown_info', inplace = True)

    return df_recruits_final

df_recruits = hs_recruits(start_year_timeframe, end_year_timeframe)

In [7]:
df_recruits.head() 

,index,id,athlete_id,recruit_type,year,ranking,name,school,committed_to,position,height,weight,stars,rating,city,state_province,country,latitude,longitude
0,0,66928,3915192.0,HighSchool,2015,1.0,Trenton Thompson,Westover,Georgia,DT,74.0,313.0,5,0.9992,Albany,GA,USA,31.578206,-84.155681
1,1,31860,NaN,HighSchool,2015,1.0,Trent Thompson,Westover,Georgia,DT,74.5,313.0,5,0.9991,Albany,GA,USA,31.578206,-84.155681
2,2,31861,-1009710.0,HighSchool,2015,2.0,Martez Ivey,Apopka,Florida,OT,77.5,275.0,5,0.9990,Apopka,FL,USA,28.677968,-81.511521
3,3,31862,3916922.0,HighSchool,2015,3.0,Byron Cowart,Armwood,Auburn,SDE,76.0,250.0,5,0.9987,Seffner,FL,USA,27.998541,-82.274884
4,4,31863,3912545.0,HighSchool,2015,4.0,Iman Marshall,Long Beach Poly,USC,CB,73.0,190.0,5,0.9985,Long Beach,CA,USA,33.769016,-118.191605


## Get Datasource 2 - College Football Team 

Get every college football team, some attributes, and their name

In [8]:
def team_dataset():

    teams_api = cfbd.TeamsApi(api_config)
    teams = teams_api.get_fbs_teams()

    df_teams = pd.DataFrame.from_records([t.to_dict() for t in teams])
    df_teams
    df_teams = df_teams[['id', 'school', 'conference', 'division', 'color', 'logos']]
    
    return df_teams

df_teams = team_dataset()

# Remove brackets around image url
df_teams['logos'] = df_teams['logos'].str.get(0)


In [9]:
df_teams.head()

,id,school,conference,division,color,logos
0,2005,Air Force,Mountain West,Mountain,#004a7b,http://a.espncdn.com/i/teamlogos/ncaa/500/2005...
1,2006,Akron,Mid-American,East,#00285e,http://a.espncdn.com/i/teamlogos/ncaa/500/2006...
2,333,Alabama,SEC,West,#690014,http://a.espncdn.com/i/teamlogos/ncaa/500/333.png
3,2026,Appalachian State,Sun Belt,East,#000000,http://a.espncdn.com/i/teamlogos/ncaa/500/2026...
4,12,Arizona,Pac-12,None,#002449,http://a.espncdn.com/i/teamlogos/ncaa/500/12.png


## Get Datasource 3 - All College Football Games 

Get every college football game played over a timeframe and stored in a dataframe

In [10]:
def games_non_transformed(start_year, final_year):
    
    
    """
    Connect to the games api and get every post and regular season game over a given time frame
    
    1) Beginning with post season, iterate over every year in given range.
    2) Union each year together
    3) Repeat for regular season
    4) Union post and regular season dfs together
    """

    # Connect to games api
    games_api = cfbd.GamesApi(api_config)

    # Post Season Games
    postseason_games = []
    for i in range(start_year, final_year + 1):

        games = games_api.get_games(year=i, season_type = 'postseason')
        df_games_post_i = pd.DataFrame.from_records([g.to_dict() for g in games])
        postseason_games.append(df_games_post_i)

    postseason_games_df = pd.concat(postseason_games)

    # Regular Season Games
    regseason_games = []
    for i in range(start_year, final_year + 1):

        games = games_api.get_games(year=i, season_type = 'regular')
        df_games_reg_i = pd.DataFrame.from_records([g.to_dict() for g in games])
        regseason_games.append(df_games_reg_i)

    regseason_games_df = pd.concat(regseason_games)

    # Union post and regular season
    return pd.concat([regseason_games_df, postseason_games_df])

df_games = games_non_transformed(start_year_timeframe, end_year_timeframe)

In [11]:
df_games.head() 

,id,season,week,season_type,start_date,start_time_tbd,completed,neutral_site,conference_game,attendance,venue_id,venue,home_id,home_team,home_conference,home_division,home_points,home_line_scores,home_post_win_prob,home_pregame_elo,home_postgame_elo,away_id,away_team,away_conference,away_division,away_points,away_line_scores,away_post_win_prob,away_pregame_elo,away_postgame_elo,excitement_index,highlights,notes
0,400795501,2015,1,regular,2015-08-29T19:30:00.000Z,False,True,False,False,NaN,3987.0,Washington-Grizzly Stadium,149,Montana,Big Sky,fcs,38.0,"[10, 11, 7, 10]",NaN,NaN,NaN,2449,North Dakota State,MVFC,fcs,35.0,"[7, 21, 0, 7]",NaN,NaN,NaN,NaN,None,None
1,400763593,2015,1,regular,2015-09-03T22:00:00.000Z,None,True,False,False,39184.0,3652.0,Bright House Networks Stadium,2116,UCF,American Athletic,fbs,14.0,"[7, 7, 0, 0]",0.101317,1626.0,1609.0,2229,Florida International,Conference USA,fbs,15.0,"[3, 0, 6, 6]",0.898683,1321.0,1338.0,5.390338,None,None
2,400603840,2015,1,regular,2015-09-03T22:00:00.000Z,None,True,True,False,51664.0,3628.0,Bank of America Stadium,2579,South Carolina,SEC,fbs,17.0,"[0, 10, 0, 7]",0.322459,1646.0,1646.0,153,North Carolina,ACC,fbs,13.0,"[7, 6, 0, 0]",0.677541,1477.0,1477.0,7.949786,None,None
3,400795587,2015,1,regular,2015-09-03T23:00:00.000Z,False,True,False,False,NaN,3658.0,Buccaneer Field,2127,Charleston Southern,Big South,fcs,41.0,"[13, 7, 14, 7]",NaN,NaN,NaN,2822,North Greenville,Independent DII,ii,14.0,"[0, 7, 7, 0]",NaN,NaN,NaN,NaN,None,None
4,400763399,2015,1,regular,2015-09-03T23:00:00.000Z,None,True,False,False,19717.0,3786.0,Kelly/Shorts Stadium,2117,Central Michigan,Mid-American,fbs,13.0,"[3, 3, 7, 0]",0.086295,1417.0,1407.0,197,Oklahoma State,Big 12,fbs,24.0,"[3, 7, 7, 7]",0.913705,1567.0,1577.0,3.188541,None,None


## Get Datasource 4 - Team Roster

Get every player on the roster

In [12]:
# df_games = games_non_transformed(start_year_timeframe, end_year_timeframe)

def team_roster(start_year, end_year):
    
    """
    Connect to the roster api and get every player on every team for every year
    """
    
    # Establish connection to Teams API
    roster_api = cfbd.TeamsApi(api_config)

    # Initiate empty list
    team_roster = []

    for i in range(start_year, end_year + 1):

        rost = roster_api.get_roster(year = i)

        # Convert list of dictionaries into a df
        df_rost = pd.DataFrame.from_records([r.to_dict() for r in rost])

        # Add an extra column specifying the season
        df_rost['year'] = i

        team_roster.append(df_rost)

    roster_df = pd.concat(team_roster)
    
    return roster_df

df_roster = team_roster(start_year_timeframe, end_year_timeframe)

## Get Datasource 5 - Composite Talent Rating

For each team and year, get a talent rating

In [13]:
# df_games = games_non_transformed(start_year_timeframe, end_year_timeframe)

def team_talent(start_year, end_year):
    
    """
    Get the talent composite rating for every single team.
    """
    
    # Establish connection to Teams API
    roster_api = cfbd.TeamsApi(api_config)

    # Initiate empty list
    team_talent = []

    for i in range(start_year, end_year + 1):

        talent = roster_api.get_talent(year = i)

        # Convert list of dictionaries into a df
        df_talent = pd.DataFrame.from_records([t.to_dict() for t in talent])

        # Add an extra column specifying the season
        df_talent['year'] = i

        team_talent.append(df_talent)

    talent_df = pd.concat(team_talent)
    
    return talent_df

# Team composite rankings only go back to 2015
# Will throw error if we try to pull data pre-2015

df_talent = team_talent(start_year_timeframe, end_year_timeframe)

#### 7. Maniuplate data so it's at the team-game grain, rather than game grain
###### -- There will be duplicate games, but we can filter for a team one one column now. 

###### -- Each game will have two records: one for the home team's perspective, one for the away team.

In [14]:
def games_manipulation(df_games):
    
    """
    The function takes in the output of the previous function games_non_transformed().
    It converts the grain of the data. Before, the grain was 1 row per game.
    Now, each game has two rows: one from the perspective of each team.
    For example, when Ohio lost to Michigan 42-27, Michigan will a win by 15 points, and OSU will show a loss by -15 points.
    
    1) First identifies every team that played at least 1 home game.
    2) Loops over every team.
    3) Converts a bunch of data points so that the numbers are referenced from the perspective of the team of interst (also called main_team)
    """


    df_seasons = []
    teams_list = list(df_games['home_team'].unique()[1:])

    # Loop over every team that played a game over the time frame specified in games_non_transformed()
    for team in teams_list:

        # find every home / away game for team of interst
        df_home = df_games[df_games['home_team'] == team]
        df_away = df_games[df_games['away_team'] == team]

        # Combine home and away games into 1 table. 
        df_season_i = pd.concat([df_home, df_away])

        # Add column specifying what team that row of data pertains to
        df_season_i['main_team'] = team

        ############
        # Adjust key columns so they represent our team of interest

        df_season_i['home_game_flag'] = np.where(df_season_i['home_team'] == team, 1, 0)

        df_season_i['team_id'] = np.where(df_season_i['home_team'] == team, df_season_i['home_id'], df_season_i['away_id'])
        df_season_i['opposing_team_id'] = np.where(df_season_i['home_team'] == team, df_season_i['away_id'], df_season_i['home_id'])

        df_season_i['team_conference'] = np.where(df_season_i['home_team'] == team, df_season_i['home_conference'], df_season_i['away_conference'])
        df_season_i['opposing_conference'] = np.where(df_season_i['home_team'] == team, df_season_i['away_conference'], df_season_i['home_conference'])

        df_season_i['points_for'] = np.where(df_season_i['home_team'] == team, df_season_i['home_points'], df_season_i['away_points'])
        df_season_i['points_against'] = np.where(df_season_i['home_team']== team, df_season_i['away_points'], df_season_i['home_points'])

        df_season_i['point_differential'] = df_season_i['points_for'] - df_season_i['points_against']

        df_season_i['team_line_scores']  = np.where(df_season_i['home_team'] == team, df_season_i['home_line_scores'], df_season_i['away_line_scores'])
        df_season_i['opposing_line_scores']  = np.where(df_season_i['home_team'] == team, df_season_i['away_line_scores'], df_season_i['home_line_scores'])

        df_season_i['team_pregame_elo']  = np.where(df_season_i['home_team'] == team, df_season_i['home_pregame_elo'], df_season_i['away_pregame_elo'])
        df_season_i['team_postgame_elo']  = np.where(df_season_i['home_team'] == team, df_season_i['home_postgame_elo'], df_season_i['away_postgame_elo'])

        df_season_i['opponent_pregame_elo'] = np.where(df_season_i['home_team'] != team, df_season_i['home_pregame_elo'], df_season_i['away_pregame_elo'])
        df_season_i['opponent_postgame_elo'] = np.where(df_season_i['home_team'] != team, df_season_i['home_postgame_elo'], df_season_i['away_postgame_elo'])

        df_season_i['win_flag'] = np.where(df_season_i['point_differential'] > 0, 1, 0)
        
#         a = df_season_i.sort_values('start_date', ascending=True) \
#                        .groupby(['main_team', 'season']) \
#                        .cumcount() + 1
        
#         df_season_i['game_that_season'] = list(a)
        
        
        ############

        df_seasons.append(df_season_i)
        
    data = pd.concat(df_seasons)
    data = data.drop(columns = ['home_id', 'home_team', 'home_conference', 'home_division', 'home_points', 'home_line_scores', 'home_post_win_prob',
                                'home_pregame_elo', 'home_postgame_elo', 'away_id', 'away_team', 'away_conference', 'away_division', 'away_points',
                                'away_line_scores', 'away_post_win_prob', 'away_pregame_elo', 'away_postgame_elo'], axis = 1)
    
    # Field that counts what game (ie the 15th game, 3rd game, etc)
    data = data.reset_index()
    data['game_that_season'] = data.sort_values(['season', 'start_date', 'team_id'], ascending=True) \
                                        .groupby(['team_id', 'season']) \
                                        .cumcount() + 1
    
    
    
    return data
    
df_manipulated_games = games_manipulation(df_games)

df_manipulated_games.head()

,index,id,season,week,season_type,start_date,start_time_tbd,completed,neutral_site,conference_game,attendance,venue_id,venue,excitement_index,highlights,notes,main_team,home_game_flag,team_id,opposing_team_id,team_conference,opposing_conference,points_for,points_against,point_differential,team_line_scores,opposing_line_scores,team_pregame_elo,team_postgame_elo,opponent_pregame_elo,opponent_postgame_elo,win_flag,game_that_season
0,1,400763593,2015,1,regular,2015-09-03T22:00:00.000Z,None,True,False,False,39184.0,3652.0,Bright House Networks Stadium,5.390338,None,None,UCF,1,2116,2229,American Athletic,Conference USA,14.0,15.0,-1.0,"[7, 7, 0, 0]","[3, 0, 6, 6]",1626.0,1609.0,1321.0,1338.0,0,1
1,321,400764868,2015,3,regular,2015-09-19T22:00:00.000Z,False,True,False,False,36484.0,3652.0,Bright House Networks Stadium,5.034525,None,None,UCF,1,2116,231,American Athletic,Southern,15.0,16.0,-1.0,"[7, 5, 3, 0]","[0, 7, 6, 3]",NaN,NaN,NaN,NaN,0,3
2,656,400764884,2015,6,regular,2015-10-10T19:45:00.000Z,None,True,False,True,26669.0,3652.0,Bright House Networks Stadium,3.008708,None,None,UCF,1,2116,41,American Athletic,American Athletic,13.0,40.0,-27.0,"[3, 0, 0, 10]","[9, 14, 17, 0]",1460.0,1360.0,1262.0,1362.0,0,6
3,824,400764895,2015,8,regular,2015-10-24T16:00:00.000Z,None,True,False,True,28350.0,3652.0,Bright House Networks Stadium,2.097159,None,None,UCF,1,2116,248,American Athletic,American Athletic,10.0,59.0,-49.0,"[7, 3, 0, 0]","[7, 17, 28, 7]",1356.0,1255.0,1712.0,1813.0,0,8
4,1276,400764911,2015,12,regular,2015-11-20T00:30:00.000Z,None,True,False,True,23734.0,3652.0,Bright House Networks Stadium,1.613710,None,None,UCF,1,2116,151,American Athletic,American Athletic,7.0,44.0,-37.0,"[7, 0, 0, 0]","[14, 17, 13, 0]",1147.0,1082.0,1479.0,1544.0,0,11


## Output Data into CSV files 

Located under the data folder. 

In [15]:
df_games.to_csv('../data/games.csv') 
df_recruits.to_csv('../data/recruits.csv')
df_teams.to_csv('../data/teams.csv')
df_manipulated_games.to_csv('../data/games_manipulated.csv')
df_roster.to_csv('../data/roster.csv')

In [16]:
print(f'Games: {df_games.shape}\n')
print(f'Game Manipulated: {df_manipulated_games.shape}\n')
print(f'Recruits: {df_recruits.shape}\n')
print(f'Teams: {df_teams.shape}\n')
print(f'Teams: {df_manipulated_games.shape}\n')
print(f'Roster: {df_roster.shape}\n')

Games: (18307, 33)

Game Manipulated: (36354, 33)

Recruits: (31874, 19)

Teams: (133, 6)

Teams: (36354, 33)

Roster: (161383, 16)

